In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("source_data/nasa_exoplanet_data_where_no_of_hoststar_greater_than_1.csv")

In [3]:
system_names = []

In [4]:
for index, data in df.iterrows():
    if data.hostname not in system_names:
        system_names.append(data.hostname)

In [13]:
for k in range(len(system_names)):
    i = system_names[k]
    if i[-1] in ['A', 'B', 'S', 'N', 'C', 'Aa', 'AB', ' ']:
        system_names[k] = system_names[k][:-1]
        

In [17]:
system_df['System'] = pd.DataFrame(system_names)

In [30]:
system_df

,System,Type,Gaia_ID
0,11 Com,G8+IIIFe-1,Gaia DR3 3946945413106333696
1,16 Cyg,,
2,18 Del,G6III,Gaia DR3 1756741374681702784
3,2MASS J01033563-5515561,M5.0,
4,2MASS J19383260+4603591,sdB+dM,Gaia DR3 2080063931448749824
...,...,...,...
340,ome Ser,K0IV,Gaia DR3 4423234230853333632
341,omi UMa,G5III:,Gaia DR3 1041808368494264576
342,psi 1 Dra,,
343,tau Boo,F7IV-V,Gaia DR3 1244571953471006720


In [24]:
from astroquery.simbad import Simbad
customSimbad = Simbad() 
customSimbad.add_votable_fields('sptype')

In [25]:
system_df['Type'] = ""

In [26]:
for i, data in system_df.iterrows():
    result = customSimbad.query_object(data.System)
    if result:
        system_df['Type'][i] = result['SP_TYPE'][0]
    else:
        system_df['Type'][i] = ''

C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : KMT-2021-BLG-0322l
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'OGLE-2007-BLG-349L' this identifier has an incorrect format for catalog: 	OGLE : Optical Gravitational Lensing Event
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'OGLE-2008-BLG-092L' this identifier has an incorrect format for catalog: 	OGLE : Optical Gravitational Lensing Event

In [27]:
system_df['Gaia_ID'] = ""

In [29]:
for i, data in system_df.iterrows():
    id_result = Simbad.query_objectids(data.System)
    if id_result:
        for x in id_result:
            if 'gaia dr3' in x['ID'].lower():
                system_df['Gaia_ID'][i] = x['ID']
    else:
        system_df['Gaia_ID'][i] = ''

C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : KMT-2021-BLG-0322l
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): 'OGLE-2007-BLG-349L' this identifier has an incorrect format for catalog: 	OGLE : Optical Gravitational Lensing Event
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): 'OGLE-2008-BLG-092L' this identifier has an incorrect format for catalog: 	OGLE : Optical Gravitational Lensing Event

In [31]:
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
gdr3 = Gaia.load_table('gaiadr3.gaia_source')

Retrieving table 'gaiadr3.gaia_source'


In [32]:
system_df['ra'] = ''
system_df['ra_error'] = ''
system_df['dec'] = ''
system_df['dec_error'] = ''
system_df['parallax'] = ''
system_df['parallax_error'] = ''
system_df['distance_gspphot'] = ''
system_df['distance_gspphot_upper'] = ''
system_df['distance_gspphot_lower'] = ''
system_df['pmra'] = ''
system_df['pmra_error'] = ''
system_df['pmdec'] = ''
system_df['pmdec_error'] = ''
system_df['pm'] = ''
system_df['radial_velocity'] = ''
system_df['radial_velocity_error'] = ''

In [34]:
for i, data in system_df.iterrows():
    query = "SELECT ra, ra_error, dec, dec_error, parallax, parallax_error, distance_gspphot, distance_gspphot_upper, distance_gspphot_lower, pmra, pmra_error, pmdec, pmdec_error, pm, radial_velocity, radial_velocity_error FROM gaiadr3.gaia_source AS gdr3 WHERE gdr3.designation = '" + data.Gaia_ID + "'"
    job = Gaia.launch_job(query)
    result = job.get_results()
    if result:
        system_df['ra'][i] = result[0]['ra']
        system_df['ra_error'][i] = result[0]['ra_error']
        system_df['dec'][i] = result[0]['dec']
        system_df['dec_error'][i] = result[0]['dec_error']
        system_df['parallax'][i] = result[0]['parallax']
        system_df['parallax_error'][i] = result[0]['parallax_error']
        system_df['distance_gspphot'][i] = result[0]['distance_gspphot']
        system_df['distance_gspphot_upper'][i] = result[0]['distance_gspphot_upper']
        system_df['distance_gspphot_lower'][i] = result[0]['distance_gspphot_lower']
        system_df['pmra'][i] = result[0]['pmra']
        system_df['pmra_error'][i] = result[0]['pmra_error']
        system_df['pmdec'][i] = result[0]['pmdec']
        system_df['pmdec_error'][i] = result[0]['pmdec_error']
        system_df['pm'][i] = result[0]['pm']
        system_df['radial_velocity'][i] = result[0]['radial_velocity']
        system_df['radial_velocity_error'][i] = result[0]['radial_velocity_error']
    else:
        system_df['ra'][i] = ''
        system_df['ra_error'][i] = ''
        system_df['dec'][i] = ''
        system_df['dec_error'][i] = ''
        system_df['parallax'][i] = ''
        system_df['parallax_error'][i] = ''
        system_df['distance_gspphot'][i] = ''
        system_df['distance_gspphot_upper'][i] = ''
        system_df['distance_gspphot_lower'][i] = ''
        system_df['pmra'][i] = ''
        system_df['pmra_error'][i] = ''
        system_df['pmdec'][i] = ''
        system_df['pmdec_error'][i] = ''
        system_df['pm'][i] = ''
        system_df['radial_velocity'][i] = ''
        system_df['radial_velocity_error'][i] = ''

In [35]:
system_df

,System,Type,Gaia_ID,ra,ra_error,dec,dec_error,parallax,parallax_error,distance_gspphot,distance_gspphot_upper,distance_gspphot_lower,pmra,pmra_error,pmdec,pmdec_error,pm,radial_velocity,radial_velocity_error
0,11 Com,G8+IIIFe-1,Gaia DR3 3946945413106333696,185.178763,0.10266,17.793264,0.10379,10.16768,0.136477,100.691399,105.460098,91.518402,-109.425802,0.14367,88.929919,0.10673,141.005447,43.251984,0.139289
1,16 Cyg,,,,,,,,,,,,,,,,,,
2,18 Del,G6III,Gaia DR3 1756741374681702784,314.60783,0.057461,10.839133,0.040246,13.302135,0.065184,--,--,--,-49.920763,0.070463,-34.508749,0.043686,60.687202,4.171299,0.11649
3,2MASS J01033563-5515561,M5.0,,,,,,,,,,,,,,,,,
4,2MASS J19383260+4603591,sdB+dM,Gaia DR3 2080063931448749824,294.635918,0.027756,46.066407,0.031607,2.440961,0.031625,--,--,--,5.225044,0.037355,-4.404689,0.042169,6.833913,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,ome Ser,K0IV,Gaia DR3 4423234230853333632,237.573246,0.058375,2.196297,0.05078,13.102552,0.069527,74.571999,74.942101,74.246696,30.263545,0.070955,-47.592899,0.058253,56.400055,-3.849087,0.122629
341,omi UMa,G5III:,Gaia DR3 1041808368494264576,127.564911,0.08631,60.71769,0.088616,17.933453,0.145647,59.303398,59.437698,59.170898,-133.643561,0.090977,-107.664451,0.112253,171.616531,--,--
342,psi 1 Dra,,,,,,,,,,,,,,,,,,
343,tau Boo,F7IV-V,Gaia DR3 1244571953471006720,206.813391,0.06893,17.457174,0.064278,64.046976,0.109317,15.6162,15.6554,15.5681,-468.92256,0.094828,63.468581,0.073549,473.198303,-16.750475,0.153261


In [36]:
system_df.to_csv("system.csv", sep=',', encoding='utf-8')